In [2]:
import pandas as pd
import numpy as np
import mysql.connector
import random
import csv
import json

In [44]:
class Episode:
    def __init__(self, id, year):
        self.id = id
        self.year = year
        self.has_recipe = {
            "recipe_name": [], 
            "recipe_cuisine_name": [],
            "chef_id": []
        }
        self.judges = []
        self.grading = {
            "judge_chef_id": [],
            "participant_chef_id": [],
            "grade": []
        }

    def get_winner(self):
        total_grade = {}
        for i in range(0, len(self.grading["grade"])):
            par = self.grading["participant_chef_id"][i]
            g = self.grading["grade"][i]
            if par in total_grade:
                total_grade[par] += g
            else:
                total_grade[par] = g
        return sorted(total_grade.items(), key=lambda x:x[1])[-1]
    
    def get_has_recipe_df(self):
        hr = {
            "episode_id": self.__get_epid(len(self.has_recipe["recipe_name"])),
            "episode_year": self.__get_epyear(len(self.has_recipe["recipe_name"])),
            "recipe_name": self.has_recipe["recipe_name"],
            "recipe_cuisine_name": self.has_recipe["recipe_cuisine_name"],
            "chef_id": self.has_recipe["chef_id"]
        }
        return pd.DataFrame(hr)

    def get_grading_df(self):
        gr = {
            "episode_id": self.__get_epid(len(self.grading["judge_chef_id"])),
            "episode_year": self.__get_epyear(len(self.grading["judge_chef_id"])),
            "judge_chef_id": self.grading["judge_chef_id"],
            "participant_chef_id": self.grading["participant_chef_id"],
            "grade": self.grading["grade"]
        }
        return pd.DataFrame(gr)

    def get_judges_df(self):
        ju = {
            "episode_id": self.__get_epid(len(self.judges)),
            "episode_year": self.__get_epyear(len(self.judges)),
            "chef_id": self.judges
        }
        return pd.DataFrame(ju)

    def __get_epid(self, length):
        epid = []
        for i in range(0, length): 
            epid.append(self.id)
        return epid
    
    def __get_epyear(self, length):
        epyear = []
        for i in range(0, length):
            epyear.append(self.year)
        return epyear

class Competition:
    def __init__(self, year, cursor):
        self.year = year
        self.cursor = cursor
        self.episodes = []

    def make_episodes(self):
        all_cuisines = self.__get_cuisines()
        recipe_has_chef = self.__get_recipe_has_chef()

        repeat_cui = {}
        repeat_chef = {}
        repeat_recipe = {}
        repeat_judge = {}
        
        for episode_id in range(0, 10):
            available_cuisines = all_cuisines[:] # copy without reference
            
            # If cuisine has been selected 3 times in a row then it's not available
            for cui in repeat_cui.keys():
                if repeat_cui[cui] >= 3:
                    available_cuisines.remove(cui)
                    repeat_cui[cui] = 0
            
            sel_cuisines = random.sample(all_cuisines, 10)     
            
            sel_chefs = []
    
            #rows = []
            ep = Episode(episode_id+1, self.year)

            available_rhc = recipe_has_chef[:]
            for chef in repeat_chef.keys():
                if repeat_chef[chef] >= 3:  
                    for x in available_rhc:
                        if chef == x[2]:
                            available_rhc.remove(x)
                    repeat_chef[chef] = 0
            for recipe in repeat_recipe.keys():
                if repeat_recipe[recipe] >= 3:
                    for x in available_rhc:
                        if recipe == x[0]:
                            available_rhc.remove(x)
                    repeat_recipe[recipe] = 0
            
            for cui in sel_cuisines:
                if cui in repeat_cui.keys():
                    repeat_cui[cui] += 1
                else:
                    repeat_cui[cui] = 1
                
                available_tuples = []
                for x in recipe_has_chef:
                    if x[1] == cui and not(x[2] in sel_chefs):
                        available_tuples.append((x[0], x[2]))
                sel_tuple = random.choice(available_tuples)
                sel_chefs.append(sel_tuple[1])

                if sel_tuple[0] in repeat_recipe.keys():
                    repeat_recipe[sel_tuple[0]] += 1
                else:
                    repeat_recipe[sel_tuple[0]] = 1

                if sel_tuple[1] in repeat_chef.keys():
                    repeat_chef[sel_tuple[1]] += 1
                else:
                    repeat_chef[sel_tuple[1]] = 1
                
                #rows.append((sel_tuple[0], cui, sel_tuple[1]))
                ep.has_recipe["recipe_name"].append(sel_tuple[0])
                ep.has_recipe["recipe_cuisine_name"].append(cui)
                ep.has_recipe["chef_id"].append(sel_tuple[1])
    
            #print(sel_cuisines)
            #print(rows)
            
            all_chef_ids = self.__get_chef_id()
            # remove chefs that are participants
            for sel_id in sel_chefs:
                all_chef_ids.remove(sel_id)
            for repeat_id in repeat_judge.keys():
                if repeat_judge[repeat_id] >= 3:
                    all_chef_ids.remove(repeat_id)
                    repeat_judge[repeat_id] = 0
            ep.judges = random.sample(all_chef_ids, 3)
    
            for judge_id in ep.judges:
                if judge_id in repeat_judge.keys():
                    repeat_judge[judge_id] += 1
                else:
                    repeat_judge[judge_id] = 1
                
                for participant_id in ep.has_recipe["chef_id"]:
                    ep.grading["judge_chef_id"].append(judge_id)
                    ep.grading["participant_chef_id"].append(participant_id)
                    ep.grading["grade"].append(random.randint(1, 5))
            self.episodes.append(ep)

            for ju in repeat_judge.keys():
                if not(ju in ep.judges):
                    repeat_judge[ju] = 0
            for cui in repeat_cui.keys():
                if not(cui in sel_cuisines):
                    repeat_cui[cui] = 0
            for chef in repeat_chef.keys():
                if not(chef in ep.has_recipe["chef_id"]):
                    repeat_chef[chef] = 0
            for recipe in repeat_recipe.keys():
                if not(recipe in ep.has_recipe["recipe_name"]):
                    repeat_recipe[recipe] = 0
        #print(repeat_judge)
        #print(repeat_cui)
        #print(repeat_chef)
        #print(repeat_recipe)

    def get_winners(self):
        winners = []
        for ep in self.episodes:
            (chef_id, grade) = ep.get_winner()
            winners.append((self.__get_chef_full_name(chef_id), chef_id, grade))
        return winners

    def __get_chef_full_name(self, chef_id):
        self.cursor.execute("select first_name, last_name from chef where id={0}".format(chef_id))
        result = self.cursor.fetchone()
        return "{0} {1}".format(result[0], result[1])
    
    def __get_cuisines(self):
        self.cursor.execute("select name from cuisine")
        cuisines = []
        for result in self.cursor:
            cuisines.append(result[0])
        return cuisines

    def __get_chef_id(self):
        self.cursor.execute("select id from chef")
        result = []
        for x in self.cursor:
            result.append(x[0])
        return result
    
    def __get_recipe_has_chef(self):
        self.cursor.execute("select * from recipe_has_chef")
        result = []
        for x in self.cursor: # x = (recipe_name, recipe_cuisine_name, chef_id)
            result.append(x)
            #print(x)
        return result

In [46]:
class App:
    def __init__(self, mycursor, db_name):
        self.mycursor = mycursor
        self.db_name = db_name
        self.ep_dict = {
            "id": [],
            "year": [],
            "image_id": []
        }
        self.ehc_frames = []
        self.ju_frames = []
        self.gr_frames = []

        self.ehc_dict = {}
        self.ju_dict = {}
        self.gr_dict = {}

    def draw(self, years):
        self.__run_app(years)
        self.__generate_dml()
        self.__generate_csv_dml()
    
    def __run_app(self, years):
        for year in years:
            comp = Competition(year, self.mycursor)
            comp.make_episodes()
            winners = comp.get_winners()
            
            eps = comp.episodes
            i = 0
            for ep in eps:
                self.ep_dict["id"].append(ep.id)
                self.ep_dict["year"].append(ep.year)
                self.ep_dict["image_id"].append(0)
                self.ehc_frames.append(ep.get_has_recipe_df())
                self.ju_frames.append(ep.get_judges_df())
                self.gr_frames.append(ep.get_grading_df())

                self.__print_episode_winner(ep.id, ep.year, winners[i])
                i+=1
                
        pd.DataFrame(self.ep_dict).to_csv("generated/episode.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
        
        ehc_df = pd.concat(self.ehc_frames) 
        ehc_df.to_csv("generated/episode_has_recipe.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
        self.ehc_dict = ehc_df.to_dict(orient='list')
        #print(self.ehc_dict)

        ju_df = pd.concat(self.ju_frames)
        ju_df.to_csv("generated/episode_judges.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
        self.ju_dict = ju_df.to_dict(orient='list')

        gr_df = pd.concat(self.gr_frames)
        gr_df.to_csv("generated/grading.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
        self.gr_dict = gr_df.to_dict(orient='list')

    def __print_episode_winner(self, epid, epyear, winner):
            print("Episode {0}.{1} winner is {2}({3}) with total grade: {4}".format(epyear, epid, winner[0], winner[1], winner[2]))
            
    def __generate_dml(self):
        f = open("generated/episode_dml.sql", "w")
        f.write("USE {0};\n".format(self.db_name))
        f.write("SET SESSION sql_mode='NO_AUTO_VALUE_ON_ZERO';\n")

        ep_values = self.__get_ep_values()
        #print(len(ep_values))
        sql = "INSERT INTO episode (id, year, image_id) VALUES (%s, %s, %s);\n"
        #print(sql % ep_values[0])
        for x in ep_values:
            f.write(sql % x)
            
        ju_values = self.__get_ju_values()
        #print(len(ju_values))
        sql = "INSERT INTO episode_judges (episode_id, episode_year, chef_id) VALUES (%s, %s, %s);\n"
        #self.mycursor.executemany(sql, ju_values)
        for x in ju_values:
            f.write(sql % x)
        
        ehc_values = self.__get_ehc_values()
        #print(len(ehc_values))
        sql = "INSERT INTO episode_has_recipe (episode_id, episode_year, recipe_name, recipe_cuisine_name, chef_id) VALUES (%s, %s, '%s', '%s', %s);\n"
        #self.mycursor.executemany(sql, ehc_values)
        for x in ehc_values:
            f.write(sql % x)
        
        gr_values = self.__get_gr_values()
        #print(len(gr_values))
        sql = "INSERT INTO grading (episode_id, episode_year, judge_chef_id, participant_chef_id, grade) VALUES (%s, %s, %s, %s, %s);\n"
        #self.mycursor.executemany(sql, gr_values)
        for x in gr_values:
            f.write(sql % x)
        f.close()

    def __generate_csv_dml(self):
        path = "/var/lib/mysql-files/data/to_import/"

        # take order from ddl
        table_order = ["episode", "episode_judges", "episode_has_recipe", "grading"]
        
        queries = ["USE {0};".format(self.db_name), 
                   "SELECT @@secure_file_priv;",
                   "SET SESSION sql_mode='NO_AUTO_VALUE_ON_ZERO';"]

        for table in table_order:
            queries.append("LOAD DATA INFILE '{0}{1}.csv' INTO TABLE {1} FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\n' IGNORE 1 ROWS;".format(path, table))

        f = open("generated/episode_csv_dml.sql", "w")
        for query in queries:
            f.write("{0}\n".format(query))
        f.close()
    
    def __get_ep_values(self):
        ep_values = []
        for i in range(0, len(self.ep_dict["id"])):
            ep_values.append((self.ep_dict["id"][i], self.ep_dict["year"][i], self.ep_dict["image_id"][i]))
        return ep_values

    def __get_ehc_values(self):
        ehc_values = []
        for i in range(0, len(self.ehc_dict["episode_id"])):
            ehc_values.append((self.ehc_dict["episode_id"][i],
                               self.ehc_dict["episode_year"][i],
                               self.ehc_dict["recipe_name"][i],
                               self.ehc_dict["recipe_cuisine_name"][i],
                               self.ehc_dict["chef_id"][i]))
        return ehc_values

    def __get_ju_values(self):
        ju_values = []
        for i in range(0, len(self.ju_dict["episode_id"])):
            ju_values.append((self.ju_dict["episode_id"][i],
                              self.ju_dict["episode_year"][i],
                              self.ju_dict["chef_id"][i]))
        return ju_values

    def __get_gr_values(self):
        gr_values = []
        for i in range(0, len(self.gr_dict["episode_id"])):
            gr_values.append((self.gr_dict["episode_id"][i],
                              self.gr_dict["episode_year"][i],
                              self.gr_dict["judge_chef_id"][i],
                              self.gr_dict["participant_chef_id"][i],
                              self.gr_dict["grade"][i]))
        return gr_values
            

In [47]:
config = open("config.json", "r")
config_data = json.load(config)

# Establish connection
mydb = mysql.connector.connect(
    host = config_data["host"],
    user = config_data["user"],
    password = config_data["password"],
    database = config_data["database"]
)
mycursor = mydb.cursor()

years = [2022, 2023, 2024]

app = App(mycursor, mydb.database)
app.draw(years)

config.close()
mycursor.close()
mydb.close()

Episode 2022.1 winner is Tiffany Parker(56) with total grade: 13
Episode 2022.2 winner is Jessica Garcia(2) with total grade: 14
Episode 2022.3 winner is Sara White(24) with total grade: 14
Episode 2022.4 winner is Kimberly Robertson(14) with total grade: 14
Episode 2022.5 winner is Jennifer Thompson(45) with total grade: 13
Episode 2022.6 winner is Jeff Jones(36) with total grade: 15
Episode 2022.7 winner is Jennifer Jackson(35) with total grade: 12
Episode 2022.8 winner is Hannah Castillo(10) with total grade: 12
Episode 2022.9 winner is Ashley Olson(94) with total grade: 14
Episode 2022.10 winner is Lawrence Jones(96) with total grade: 10
Episode 2023.1 winner is Douglas Thomas(69) with total grade: 13
Episode 2023.2 winner is Janet Fitzpatrick(71) with total grade: 11
Episode 2023.3 winner is Julie Fuller(17) with total grade: 14
Episode 2023.4 winner is Justin Gray(12) with total grade: 14
Episode 2023.5 winner is Anna Moore(23) with total grade: 15
Episode 2023.6 winner is Jackie